In [2]:
import site
site.addsitedir("/rhome/kzhao028/.local/lib/python3.11/site-packages")
import pandas as pd
import numpy as np
import os
import glob
import gffutils

import seaborn as sns
import scipy as scipy
import sys
import matplotlib
matplotlib.style.use('ggplot')
idx = pd.IndexSlice

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [10]:
fqlib=glob.glob('/rhome/kzhao028/shared/seqdata/20251202*/*fastq.gz')
print(fqlib[0])
bamlist=glob.glob('./bamfiles_hg38/*bam')

/rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/D25XYN_2_mCh_rep2.fastq.gz


In [13]:
import os

samplename_list = []
sname_list = []

def strip_fastq_ext(fname):
    """Remove .fastq.gz or .fq.gz but keep entire basename."""
    if fname.endswith(".fastq.gz"):
        return fname[:-9]
    if fname.endswith(".fq.gz"):
        return fname[:-6]
    return os.path.splitext(fname)[0]

# Build sample lists
for l in fqlib:
    if 'EKDL' not in l:
        base = os.path.basename(l)
        name = strip_fastq_ext(base)
        samplename_list.append(name)
        sname_list.append(name)

# Build list of existing BAM files to skip
bname_list = []
for b in bamlist:
    if 'EKDL25' not in b:
        base = os.path.basename(b)
        name = strip_fastq_ext(base)
        bname_list.append(name)

# Identify samples that still need STAR alignment
the_rest_list = list(set(sname_list).difference(bname_list))
print("Samples to process:", len(the_rest_list))
print("Existing BAMs:", len(bname_list))

# Output directory
outdir = "/rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/star_output/"
indir  = "/rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/"

# Build STAR commands for single-end reads
cmdlist = []
for sample in the_rest_list:

    fastq_path = os.path.join(indir, sample + ".fastq.gz")
    prefix = os.path.join(outdir, sample + "_")

    starcmd = (
        'STAR --genomeDir /rhome/kzhao028/shared/Joy/references/human_flu/fluWSN_nicole_hg38v47 '
        '--readFilesIn ' + fastq_path +
        ' --readFilesCommand zcat '
        '--outFileNamePrefix ' + prefix +
        ' --outSAMtype BAM SortedByCoordinate --runThreadN 16\n'
    )
     #   'mv ' + prefix + '*out* '
      #  '/rhome/jshengna/shared/Joy/20250419-IRD_rna_editing/star_output/\n'
       # )


    print(starcmd)
    cmdlist.append(starcmd)


Samples to process: 15
Existing BAMs: 0
STAR --genomeDir /rhome/kzhao028/shared/Joy/references/human_flu/fluWSN_nicole_hg38v47 --readFilesIn /rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/D25XYN_1_mCh_rep1.fastq.gz --readFilesCommand zcat --outFileNamePrefix /rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/star_output/D25XYN_1_mCh_rep1_ --outSAMtype BAM SortedByCoordinate --runThreadN 16

STAR --genomeDir /rhome/kzhao028/shared/Joy/references/human_flu/fluWSN_nicole_hg38v47 --readFilesIn /rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/D25XYN_5_PB1_rep2.fastq.gz --readFilesCommand zcat --outFileNamePrefix /rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/star_output/D25XYN_5_PB1_rep2_ --outSAMtype BAM SortedByCoordinate --runThreadN 16

STAR --genomeDir /rhome/kzhao028/shared/Joy/references/human_flu/fluWSN_nicole_hg38v47 --readFilesIn /rhome/kzhao028/shared/seqdata/20251202-Neil_Flu_RNAseq/D25XYN_2_mCh_rep2.fastq.gz --readFilesCommand zcat --outFileNamePre